<a href="https://colab.research.google.com/github/Souleymaneawe/Souleymaneawe/blob/main/seasonal_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Étape 1 : Installer les bibliothèques
!pip install gradio google-generativeai firebase-admin

# Étape 2 : Importer les bibliothèques
import gradio as gr
import google.generativeai as genai
import firebase_admin
from firebase_admin import credentials, auth, firestore
from google.colab import userdata
import json
import random

In [2]:
# Étape 3 : Configurer Firebase
firebase_app_initialized = False
db = None
try:
    if not firebase_admin._apps:
        firebase_config = {
            "type": userdata.get('FIREBASE_TYPE'),
            "project_id": userdata.get('FIREBASE_PROJECT_ID'),
            "private_key_id": userdata.get('FIREBASE_PRIVATE_KEY_ID'),
            "private_key": userdata.get('FIREBASE_PRIVATE_KEY').replace('\\n', '\n'),
            "client_email": userdata.get('FIREBASE_CLIENT_EMAIL'),
            "client_id": userdata.get('FIREBASE_CLIENT_ID'),
            "auth_uri": userdata.get('FIREBASE_AUTH_URI'),
            "token_uri": userdata.get('FIREBASE_TOKEN_URI'),
            "auth_provider_x509_cert_url": userdata.get('FIREBASE_AUTH_PROVIDER_X509_CERT_URL'),
            "client_x509_cert_url": userdata.get('FIREBASE_CLIENT_X509_CERT_URL')
        }
        cred = credentials.Certificate(firebase_config)
        firebase_admin.initialize_app(cred)
        firebase_app_initialized = True
    db = firestore.client()
except userdata.SecretNotFoundError:
    print("Erreur : Identifiants Firebase non trouvés. Ajoutez les clés dans les Secrets de Colab.\nL'authentification et la sauvegarde des préférences ne fonctionneront pas.")
except Exception as e:
    print(f"Erreur lors de l'initialisation de Firebase : {e}\nL'authentification et la sauvegarde des préférences ne fonctionneront pas.")

In [3]:
# Étape 4 : Configurer l'API Gemini
GEMINI_API_KEY = None
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
except userdata.SecretNotFoundError:
    print("Erreur : Clé API Gemini non trouvée. Ajoutez 'GEMINI_API_KEY' dans les Secrets de Colab.\nLes suggestions de plats ne fonctionneront pas.")

In [4]:
# Étape 5 : Charger les données des saisons depuis JSON
def load_seasons():
    try:
        with open("seasons.json", "r", encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print("Fichier 'seasons.json' non trouvé. Utilisation de données par défaut. Veuillez créer le fichier.\n")
        return {
            "seasons": [
    {
      "name": "Printemps",
      "description": "Saison des légumes frais et des plats légers, propice au renouveau culinaire.",
      "main_ingredients": ["asperge", "radis", "petits pois", "fraise", "rhubarbe", "oignon nouveau", "épinard", "agneau", "poule"]
    },
    {
      "name": "Été",
      "description": "Saison des fruits juteux, des légumes du soleil et des repas rafraîchissants et colorés.",
      "main_ingredients": ["tomate", "courgette", "aubergine", "poivron", "melon", "pastèque", "framboise", "myrtille", "basilic", "concombre", "pêche", "abricot"]
    },
    {
      "name": "Automne",
      "description": "Saison des récoltes riches, des champignons et des plats réconfortants et chaleureux.",
      "main_ingredients": ["potiron", "citrouille", "champignon", "châtaigne", "pomme", "poire", "raisin", "coing", "figue", "noix", "gibier", "cèpe"]
    },
    {
      "name": "Hiver",
      "description": "Saison des soupes chaudes, des légumes racines et des plats copieux et nutritifs.",
      "main_ingredients": ["poireau", "chou", "carotte", "pomme de terre", "panais", "orange", "clémentine", "kiwi", "choux de Bruxelles", "endive", "viande rouge", "fromage"]
    }
  ]
        }

In [5]:
# Étape 6 : Fonctions Firebase
def register_user(email, password):
    if not firebase_app_initialized:
        return "Erreur : Firebase non initialisé. Impossible de s'inscrire."
    if not email or not password:
        return "Veuillez fournir un email et un mot de passe."
    try:
        user = auth.create_user(email=email, password=password)
        db.collection('users').document(user.uid).set({'email': email, 'preferences': ''})
        return f"Inscription réussie pour {email}. Vous pouvez maintenant vous connecter."
    except Exception as e:
        if 'auth/email-already-exists' in str(e):
            return "Cet email est déjà enregistré."
        elif 'auth/invalid-password' in str(e):
            return "Mot de passe invalide. Doit avoir au moins 6 caractères."
        return f"Erreur lors de l'inscription : {str(e)}"

def login_user(email, password):
    if not firebase_app_initialized:
        return None, "Erreur : Firebase non initialisé. Impossible de se connecter.", ""
    if not email or not password:
        return None, "Veuillez fournir un email et un mot de passe.", ""
    try:
        user = auth.get_user_by_email(email)
        user_doc = db.collection('users').document(user.uid).get()
        preferences = user_doc.to_dict().get('preferences', '') if user_doc.exists else ''
        return user.uid, f"Connexion réussie pour {email}.", preferences
    except Exception as e:
        if 'auth/user-not-found' in str(e) or 'auth/invalid-email' in str(e):
            return None, "Email ou mot de passe incorrect.", ""
        return None, f"Erreur lors de la connexion : {str(e)}", ""

def save_preferences(uid, preferences):
    if not firebase_app_initialized or not db:
        return "Erreur : Firebase non initialisé. Impossible d'enregistrer les préférences."
    if not uid:
        return "Vous devez être connecté pour enregistrer les préférences."
    try:
        db.collection('users').document(uid).update({'preferences': preferences})
        return "Préférences enregistrées avec succès."
    except Exception as e:
        return f"Erreur lors de l'enregistrement des préférences : {str(e)}"

In [6]:
# Étape 7 : Fonction pour les suggestions de plats
def get_food_suggestions(season_name, preferences, uid, all_seasons_data):
    if not GEMINI_API_KEY:
        return "Erreur : Clé API Gemini manquante. Veuillez la configurer dans les Secrets de Colab."
    model = genai.GenerativeModel("gemini-1.5-pro")
    current_season_info = next((s for s in all_seasons_data["seasons"] if s["name"] == season_name), None)
    if current_season_info and current_season_info.get("main_ingredients"):
        ingredients = random.sample(current_season_info["main_ingredients"],
                                   min(len(current_season_info["main_ingredients"]), 3))
        ingredients_str = ", ".join(ingredients)
        seasonal_context = f" utilisant des ingrédients clés comme {ingredients_str}."
    else:
        seasonal_context = "."
    prompt = (
        f"Suggère 3 à 5 idées de plats originaux et équilibrés typiques de la saison {season_name}{seasonal_context} "
        f"Considère les préférences alimentaires suivantes : '{preferences}'. "
        f"Pour chaque plat, donne son nom, 2-3 ingrédients clés, et une courte description (1-2 phrases). "
        f"Présente les suggestions sous forme de liste numérotée, en mettant le nom du plat en gras. "
        f"Termine par une courte phrase pour inspirer l'utilisateur."
    )
    try:
        response = model.generate_content(prompt)
        if uid and preferences:
            save_preferences(uid, preferences)
        return response.text
    except Exception as e:
        return f"Erreur API Gemini : {str(e)}\nAssurez-vous que votre clé API est valide et que le modèle est disponible."

In [14]:
# Étape 8 : Interface Gradio
initial_preferences = ""
with gr.Blocks(
    title="Suggestions de plats saisonniers avec Firebase",
    css="""
        .gradio-container { background-color: #f5f7fa; font-family: 'Segoe UI', Arial, sans-serif; }
        h1 { color: #2c3e50; text-align: center; font-size: 2em; margin-bottom: 20px; }
        .gr-button { background-color: #28a745; color: white; border-radius: 10px; padding: 10px; }
        .gr-button:hover { background-color: #218838; }
        .gr-textbox, .gr-dropdown { border-radius: 10px; border: 1px solid #ced4da; padding: 8px; }
        .gr-form { background-color: #ffffff; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        #auth_status { color: red; }
        #success_status { color: green; }
    """
) as app:
    gr.Markdown("# Suggestions de plats saisonniers")
    # Section d'authentification
    with gr.Tab("Connexion / Inscription"):
        gr.Markdown("## Connexion ou Inscription")
        auth_info_md = gr.Markdown(value="", elem_id="auth_status")
        email = gr.Textbox(label="Adresse e-mail")
        password = gr.Textbox(label="Mot de passe", type="password")
        login_btn = gr.Button("Se connecter")
        register_btn = gr.Button("S'inscrire")
        uid_state = gr.State(None)
        user_preferences_state = gr.State("")

        def handle_login(email_input, password_input):
            uid, status_message, preferences = login_user(email_input, password_input)
            if uid:
                return uid, status_message, preferences, gr.update(value=preferences)
            else:
                return None, status_message, "", gr.update(value="")

        login_btn.click(
            fn=handle_login,
            inputs=[email, password],
            outputs=[uid_state, auth_info_md, user_preferences_state, preferences]
        )
        register_btn.click(
            fn=register_user,
            inputs=[email, password],
            outputs=auth_info_md
        )

    # Section des suggestions
    with gr.Tab("Suggestions de plats"):
        seasons_data = load_seasons()
        season_names = [season["name"] for season in seasons_data["seasons"]]
        gr.Markdown("## Sélectionnez une saison")
        season_dropdown = gr.Dropdown(choices=season_names, label="Saison")
        season_description = gr.Textbox(label="Description de la saison", interactive=False)

        def update_season_description(selected_season):
            for season_data_item in seasons_data["seasons"]:
                if season_data_item["name"] == selected_season:
                    return season_data_item["description"]
            return "Saison non trouvée."

        season_dropdown.change(fn=update_season_description, inputs=season_dropdown, outputs=season_description)

        preferences = gr.Textbox(label="Préférences alimentaires (ex. végétarien, sans gluten)", value=initial_preferences)
        suggest_btn = gr.Button("Obtenir des suggestions")
        output = gr.Markdown(label="Suggestions de plats")

        suggest_btn.click(
            fn=get_food_suggestions,
            inputs=[season_dropdown, preferences, uid_state, gr.State(seasons_data)],
            outputs=output
        )

    # Messages d'avertissement si les clés API sont manquantes
    if not firebase_app_initialized:
        gr.Markdown("<p style='color:red;'><b>Avertissement :</b> Les identifiants Firebase sont manquants. L'authentification et la sauvegarde des préférences ne fonctionneront pas.</p>")
    if not GEMINI_API_KEY:
        gr.Markdown("<p style='color:red;'><b>Avertissement :</b> La clé API Gemini est manquante. Les suggestions de plats ne fonctionneront pas.</p>")

NameError: name 'preferences' is not defined

In [17]:
# Étape 8 : Interface Gradio
initial_preferences = ""
with gr.Blocks(
    title="Suggestions de plats saisonniers avec Firebase",
    css="""
        .gradio-container { background-color: #f5f7fa; font-family: 'Segoe UI', Arial, sans-serif; }
        h1 { color: #2c3e50; text-align: center; font-size: 2em; margin-bottom: 20px; }
        .gr-button { background-color: #28a745; color: white; border-radius: 10px; padding: 10px; }
        .gr-button:hover { background-color: #218838; }
        .gr-textbox, .gr-dropdown { border-radius: 10px; border: 1px solid #ced4da; padding: 8px; }
        .gr-form { background-color: #ffffff; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        #auth_status { color: red; }
        #success_status { color: green; }
    """
) as app:
    gr.Markdown("# Suggestions de plats saisonniers")
    # Define preferences textbox here, accessible to all tabs
    preferences = gr.Textbox(label="Préférences alimentaires (ex. végétarien, sans gluten)", value=initial_preferences)

    # Section d'authentification
    with gr.Tab("Connexion / Inscription"):
        gr.Markdown("## Connexion ou Inscription")
        auth_info_md = gr.Markdown(value="", elem_id="auth_status")
        email = gr.Textbox(label="Adresse e-mail")
        password = gr.Textbox(label="Mot de passe", type="password")
        login_btn = gr.Button("Se connecter")
        register_btn = gr.Button("S'inscrire")
        uid_state = gr.State(None)
        user_preferences_state = gr.State("")

        def handle_login(email_input, password_input):
            uid, status_message, preferences_from_db = login_user(email_input, password_input)
            if uid:
                return uid, status_message, preferences_from_db, gr.update(value=preferences_from_db)
            else:
                return None, status_message, "", gr.update(value="")

        login_btn.click(
            fn=handle_login,
            inputs=[email, password],
            outputs=[uid_state, auth_info_md, user_preferences_state, preferences]
        )
        register_btn.click(
            fn=register_user,
            inputs=[email, password],
            outputs=auth_info_md
        )

    # Section des suggestions
    with gr.Tab("Suggestions de plats"):
        seasons_data = load_seasons()
        season_names = [season["name"] for season in seasons_data["seasons"]]
        gr.Markdown("## Sélectionnez une saison")
        season_dropdown = gr.Dropdown(choices=season_names, label="Saison")
        season_description = gr.Textbox(label="Description de la saison", interactive=False)

        def update_season_description(selected_season):
            for season_data_item in seasons_data["seasons"]:
                if season_data_item["name"] == selected_season:
                    return season_data_item["description"]
            return "Saison non trouvée."

        season_dropdown.change(fn=update_season_description, inputs=season_dropdown, outputs=season_description)

        # preferences is now defined outside the tab
        suggest_btn = gr.Button("Obtenir des suggestions")
        output = gr.Markdown(label="Suggestions de plats")

        suggest_btn.click(
            fn=get_food_suggestions,
            inputs=[season_dropdown, preferences, uid_state, gr.State(seasons_data)],
            outputs=output
        )

    # Messages d'avertissement si les clés API sont manquantes
    if not firebase_app_initialized:
        gr.Markdown("<p style='color:red;'><b>Avertissement :</b> Les identifiants Firebase sont manquants. L'authentification et la sauvegarde des préférences ne fonctionneront pas.</p>")
    if not GEMINI_API_KEY:
        gr.Markdown("<p style='color:red;'><b>Avertissement :</b> La clé API Gemini est manquante. Les suggestions de plats ne fonctionneront pas.</p>")

# To run the Gradio app, you would typically add app.launch() here or in a separate cell.
# app.launch()

In [18]:
app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75e2d1a0b03b5be153.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
